In [1]:
import pandas as pd 
import numpy as np

# Dataframe de la base de données sur les incendies: 
df_incendies = pd.read_csv(r"/Users/thomas/Desktop/projetpython/ensae-prog2A/data/Incendies.csv", sep = ';')

# Dataframe avec la base de données permettant de faire la correspondance commune-coordonnées GPS :
df_communes = pd.read_csv(r"/Users/thomas/Desktop/projetpython/ensae-prog2A/data/communes.csv")
df_communes = df_communes[['code_commune_INSEE','latitude','longitude','nom_region','nom_commune','nom_commune_complet']]
df_communes.columns = ['code_commune_INSEE', 'lat', 'lon', 'region', 'nom_commune','nom_commune_complet']

# Dataframe avec la base de données meteonet pour le quart sud-est :
df_meteo_SE = pd.read_csv(r'/Users/thomas/Desktop/projetpython/ensae-prog2A/data/SE2018.csv')

# Dataframe avec juste les stations et leurs coordonnées GPS :
df_stations_SE = df_meteo_SE[['number_sta','lat','lon']]


In [2]:
df_stations_SE = df_stations_SE.drop_duplicates()
liste_regions = ['Corse', "Provence-Alpes-Côte d'Azur", "Occitanie", "Auvergne-Rhône-Alpes", 'Nouvelle-Aquitaine']
df_communes = df_communes[df_communes['region'].isin(liste_regions)]

def distance(lat1,lon1,lat2,lon2):
    """ INPUT : les coordonnées GPS de deux points
        OUTPUT : la distance entre ces deux points à vol d'oiseau
    """
    return 6371*np.arccos(np.sin(lat1*np.pi/180)*np.sin(lat2*np.pi/180)+np.cos(lat1*np.pi/180)*np.cos(lat2*np.pi/180)*np.cos((lon2-lon1)*np.pi/180))

# créer une colonne vide "commune" :
df_stations_SE['code_commune_INSEE']=""
# créer la colonne "distance station-commune" exprimée en km, arbitrairement égale à 0 pour l'instant :
df_stations_SE['distance station-commune (km)']=0
display(df_stations_SE.head())

,number_sta,lat,lon,code_commune_INSEE,distance station-commune (km)
0,1027003,45.83,5.11,,0
1,1033002,46.09,5.81,,0
2,1034004,45.77,5.69,,0
3,1072001,46.20,5.29,,0
4,1089001,45.98,5.33,,0


In [3]:
#associer à chaque station la commune la plus proche
for i in df_stations_SE.index:
    lat1 = df_stations_SE['lat'][i]
    lon1 = df_stations_SE['lon'][i]
    s= 100 #initialise une distance arbitraire 
    plusprochecommune=0
    for j in df_communes.index:
        lat2 = df_communes['lat'][j]
        lon2 = df_communes['lon'][j]
        d = distance(lat1,lon1,lat2,lon2)
        if d <= s:
            s = d
            plusprochecommune = df_communes['code_commune_INSEE'][j]
    df_stations_SE['code_commune_INSEE'][i] = plusprochecommune
    df_stations_SE['distance station-commune (km)'][i] = s

      

/var/folders/vt/6c09fl4n7h7dmmfb0j2ctpf40000gn/T/ipykernel_15810/4056711664.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_SE['code_commune_INSEE'][i] = plusprochecommune
/var/folders/vt/6c09fl4n7h7dmmfb0j2ctpf40000gn/T/ipykernel_15810/4056711664.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_SE['distance station-commune (km)'][i] = s
/var/folders/vt/6c09fl4n7h7dmmfb0j2ctpf40000gn/T/ipykernel_15810/4056711664.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.21126365480751144' has dtype incompatible with int

In [4]:
#df_stations_SE = df_stations_SE.rename(columns = {'commune' :'code_commune_INSEE' })
#df_incendies = df_incendies.rename(columns = {'Code INSEE' : 'code_commune_INSEE' })

In [29]:
df_incendies = pd.read_csv(r"/Users/thomas/Desktop/projetpython/ensae-prog2A/data/incendies.csv", sep = ";")
df_incendies = df_incendies.rename(columns = {'Code INSEE' : 'code_commune_INSEE' })
df_incendies_18 = df_incendies[df_incendies['Année']==2018]

df_incendies_18['code_commune_INSEE']=df_incendies_18['code_commune_INSEE'].astype(str)


In [6]:
# mettre les dates dans le même format
df_meteo_SE['date'] = pd.to_datetime(df_meteo_SE['date'])
df_incendies_18['date'] = pd.to_datetime(df_incendies_18['Date de première alerte'])

In [7]:
from datetime import datetime
liste_minutes = [6*k for k in range(10)]
def date_approchee(df):
    #(df['date'])
    for el in liste_minutes :
        if df['date'].minute >= el and df['date'].minute-el < 6 :
            new_date = df['date'].replace(minute = el)
            return new_date
df_incendies_18['date'] = df_incendies_18.apply(date_approchee, axis = 1)

display(df_incendies_18.head())

,Année,Numéro,Département,code_commune_INSEE,Nom de la commune,Date de première alerte,Origine de l'alerte,Moyens de première intervention,Surface parcourue (m2),Surface forêt (m2),...,Nombre de décès,Nombre de bâtiments totalement détruits,Nombre de bâtiments partiellement détruits,Hygrométrie (%),Vitesse moyenne du vent (Km/h),Direction du vent,Température (°C),Précision de la donnée,Présence d'un contour valide,date
0,2018,3257,83,83082,Montferrat,2018-04-21 13:43:00,Population,3.0,5000,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,2018-04-21 13:42:00
1,2018,3266,6,6074,Lantosque,2018-04-23 14:05:00,Population,3.0,900,900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,2018-04-23 14:00:00
2,2018,3270,6,6074,Lantosque,2018-04-26 15:18:00,Population,3.0,1500,1500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,2018-04-26 15:18:00
3,2018,3271,6,6074,Lantosque,2018-04-26 17:04:00,Population,3.0,600,600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,2018-04-26 17:00:00
4,2018,3279,6,6130,Saint-Vallier-de-Thiey,2018-04-28 14:01:00,Population,3.0,5100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,2018-04-28 14:00:00


In [8]:
# ajout du code commune INSEE dans la base de la météo :
df_meteo_SE_new = pd.merge(df_meteo_SE, df_stations_SE[['number_sta','code_commune_INSEE']], on = 'number_sta', how = 'right')
display(df_meteo_SE_new.head(10))

#df = pd.merge(df_meteo_SE_new, df_incendies_18, on = ['code_commune_INSEE', 'date'], how = 'inner')
#display(df.head(30))

,number_sta,lat,lon,height_sta,date,dd,ff,precip,hu,td,t,psl,code_commune_INSEE
0,1027003,45.83,5.11,196.0,2018-01-01 00:00:00,NaN,NaN,NaN,90.0,279.15,280.65,NaN,1027
1,1027003,45.83,5.11,196.0,2018-01-01 00:06:00,NaN,NaN,NaN,92.0,279.45,280.65,NaN,1027
2,1027003,45.83,5.11,196.0,2018-01-01 00:12:00,NaN,NaN,NaN,92.0,279.35,280.55,NaN,1027
3,1027003,45.83,5.11,196.0,2018-01-01 00:18:00,NaN,NaN,NaN,93.0,279.35,280.45,NaN,1027
4,1027003,45.83,5.11,196.0,2018-01-01 00:24:00,NaN,NaN,NaN,92.0,279.15,280.35,NaN,1027
5,1027003,45.83,5.11,196.0,2018-01-01 00:30:00,NaN,NaN,NaN,92.0,279.05,280.25,NaN,1027
6,1027003,45.83,5.11,196.0,2018-01-01 00:36:00,NaN,NaN,NaN,94.0,279.35,280.25,NaN,1027
7,1027003,45.83,5.11,196.0,2018-01-01 00:42:00,NaN,NaN,NaN,95.0,279.45,280.15,NaN,1027
8,1027003,45.83,5.11,196.0,2018-01-01 00:48:00,NaN,NaN,NaN,96.0,279.55,280.15,NaN,1027
9,1027003,45.83,5.11,196.0,2018-01-01 00:54:00,NaN,NaN,NaN,95.0,279.35,280.05,NaN,1027


In [135]:
def meteo_commune(code): #fonction qui renvoie les données météo quotidiennes d'une commune donnée
    "mettre en input le code commune INSEE en chaîne de caractère"
    df_meteo_locale = df_meteo_SE_new[df_meteo_SE_new['code_commune_INSEE']== code]

    df_meteo_locale['jour'] = df_meteo_locale['date'].dt.date
    moyennes_quotidiennes = df_meteo_locale[['dd','ff','precip','hu','td','t','psl','jour']].groupby('jour').mean()
    moyennes_quotidiennes=moyennes_quotidiennes.rename(columns ={'dd':'dd_moyen','ff':'ff_moyen','precip':'precip_moyen','hu':'hu_moyen','td':'td_moyen','t':'t_moyen','psl':'psl_moyen',})
    #moyennes_quotidiennes = moyennes_quotidiennes.reset_index()
    
    max_quotidiens = df_meteo_locale[['dd','ff','precip','hu','td','t','psl','jour']].groupby('jour').max()
    max_quotidiens=max_quotidiens.rename(columns ={'dd':'dd_max','ff':'ff_max','precip':'precip_max','hu':'hu_max','td':'td_max','t':'t_max','psl':'psl_max',})
    #max_quotidiens=max_quotidiens.reset_index()

    ecarts_quotidiens = df_meteo_locale[['dd','ff','precip','hu','td','t','psl','jour']].groupby('jour').std()
    ecarts_quotidiens=ecarts_quotidiens.rename(columns ={'dd':'dd_std','ff':'ff_std','precip':'precip_std','hu':'hu_sd','td':'td_std','t':'t_std','psl':'psl_std',})
    #ecarts_quotidiens=ecarts_quotidiens.reset_index()
    
    df_meteo_quotidienne = pd.merge(moyennes_quotidiennes,max_quotidiens, on='jour').merge(ecarts_quotidiens, on='jour')
    return df_meteo_quotidienne




    

In [140]:
display(meteo_commune('1089').head())

pas de station dans cette commune


/var/folders/vt/6c09fl4n7h7dmmfb0j2ctpf40000gn/T/ipykernel_15810/4197168212.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meteo_locale['jour'] = df_meteo_locale['date'].dt.date


,dd_moyen,ff_moyen,precip_moyen,hu_moyen,td_moyen,t_moyen,psl_moyen,dd_max,ff_max,precip_max,...,td_max,t_max,psl_max,dd_std,ff_std,precip_std,hu_sd,td_std,t_std,psl_std
jour,,,,,,,,,,,,,,,,,,,,,
2018-01-01,174.936709,3.789030,0.010127,80.776371,277.311603,280.512447,101658.565401,350.0,8.0,0.2,...,279.75,282.55,102070.0,44.012493,2.010492,0.043896,9.732391,1.168003,1.053246,345.251768
2018-01-02,158.765957,2.648936,0.002553,83.302128,277.171277,279.920213,102131.744681,270.0,5.8,0.2,...,279.25,282.25,102470.0,46.458461,1.447367,0.022477,10.321805,1.022839,1.401083,150.112746
2018-01-03,207.564103,4.709829,0.050000,75.106838,279.361111,283.755983,101586.282051,350.0,11.2,0.8,...,282.85,287.65,101950.0,53.937530,2.100930,0.133235,10.948650,1.245104,2.477022,251.451166
2018-01-04,169.666667,3.931250,0.089583,87.891667,282.058333,284.060000,101216.333333,200.0,5.6,0.8,...,283.95,287.25,101670.0,8.662865,0.649765,0.140443,7.147472,1.618994,2.365952,123.986890
2018-01-05,153.708333,3.445833,0.000833,79.237500,281.676250,285.227917,100669.791667,360.0,5.9,0.2,...,283.05,287.55,101090.0,71.319200,0.935176,0.012896,6.698226,0.605695,1.550208,176.309767
